In [50]:
import os
import seaborn as sns
import matplotlib.pyplot as plt

import SALib.analyze.morris
from SALib.sample import morris

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../')

from tqdm import tqdm

from infrasim.optimise import *
from infrasim.utils import *

In [119]:
#File paths
nodes = '../data/nextra/spatial/network/nodes.shp'
edges = '../data/nextra/spatial/network/edges.shp'
flows = '../data/nextra/nodal_flows/processed_flows_2030.csv'

timesteps   = 24
num_levels  = 10
N           = 10

# Set up problem for sensitivity analysis # name:[lower_bound,upper_bound]
params = {'coop_res_target'                 : [0,0.5],
          'jordan_to_westbank'              : [0,10**12],
          'jordan_to_israel'                : [0,10**12],
          'israel_to_westbank'              : [0,10**12],
          'israel_to_jordan'                : [0,10**12],
          'israel_to_gaza'                  : [0,10**12],
          'westbank_to_israel'              : [0,10**12],
          'westbank_to_jordan'              : [0,10**12],
          'self_sufficiency_factor'         : [0,0.9],
         }

problem = {'num_vars'    : len(params.keys()),
           'names'       : [i for i in params.keys()],
           'bounds'      : [params[i] for i in params.keys()]
          }

# create parameter values
param_values = morris.sample(problem,N=N,
                             num_levels=num_levels,
                             local_optimization=True)

# Run analysis with the specified parameter values
collected_caps_israel   = []
collected_caps_jordan   = []
collected_caps_westbank = []
collected_caps_gaza     = []

for param_set in tqdm(param_values,total=len(param_values)):

    model_run = nextra(nodes,edges,flows,
                       timesteps=timesteps,
                       energy_objective=True,
                       scenario='COO',
                       super_sink=False,
                       super_source=False,
                       # params
                       coo_res_factor=param_set[0],
                       jordan_to_westbank=param_set[1],
                       jordan_to_israel=param_set[2],
                       israel_to_westbank=param_set[3],
                       israel_to_jordan=param_set[4],
                       israel_to_gaza=param_set[5],
                       westbank_to_israel=param_set[6],
                       westbank_to_jordan=param_set[7],
                       self_sufficiency_factor=param_set[8],
                      )
    
    # build, run, and get results
    model_run.build()
    model_run.run(pprint=False)
    model_results = model_run.get_results()
    
    # get capacities
    caps = model_results.results_capacities.groupby(by=['node','territory']).max().reset_index()
    caps = caps.groupby(by='territory').sum().reset_index()
    
    # append collected capacities
    collected_caps_israel.append(caps.loc[caps.territory == 'Israel','value'].values[0])
    collected_caps_jordan.append(caps.loc[caps.territory == 'Jordan','value'].values[0])
    collected_caps_westbank.append(caps.loc[caps.territory == 'West Bank','value'].values[0])
    collected_caps_gaza.append(caps.loc[caps.territory == 'Gaza','value'].values[0])



  1%|▉                                                                                       | 1/100 [00:01<01:47,  1.09s/it]

  2%|█▊                                                                                      | 2/100 [00:01<01:12,  1.35it/s]

  3%|██▋                                                                                     | 3/100 [00:02<01:00,  1.59it/s]

  4%|███▌                                                                                    | 4/100 [00:02<00:56,  1.71it/s]

  5%|████▍                                                                                   | 5/100 [00:03<00:53,  1.79it/s]

...

 96%|███████████████████████████████████████████████████████████████████████████████████▌   | 96/100 [00:50<00:02,  1.90it/s]

 97%|████████████████████████████████████████████████████████████████████████████████████▍  | 97/100 [00:50<00:01,  1.94it/s]

 98%|█████████████████████████████████████████████████████████████████████████████████████▎ | 98/100 [00:51<00:01,  1.96it/s]

 99%|██████████████████████████████████████████████████████████████████████████████████████▏| 99/100 [00:51<00:00,  1.89it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:52<00:00,  1.91it/s]

In [120]:
def process_morris_results(problem,param_values,collected_results,num_levels):
    '''Process output from Morris sensitivitys
    '''
    Si = SALib.analyze.morris.analyze(problem,
                                      np.array(param_values),
                                      np.array(collected_results),
                                      print_to_console=False,
                                      num_levels=num_levels)

    sensitivity_results = pd.DataFrame.from_dict(Si)
    return sensitivity_results

israel_results = process_morris_results(problem,param_values,collected_caps_israel,num_levels)
jordan_results = process_morris_results(problem,param_values,collected_caps_jordan,num_levels)
westbank_results = process_morris_results(problem,param_values,collected_caps_westbank,num_levels)
gaza_results = process_morris_results(problem,param_values,collected_caps_gaza,num_levels)